In [5]:
import ase.build
from metatomic.torch import ModelOutput
from metatomic.torch.ase_calculator import MetatomicCalculator
from ase.md.bussi import Bussi
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
import numpy as np

In [ ]:
system = ase.build.bulk("Al", "fcc", a=4.05)
system = system * (3, 3, 3)  # 108 atoms
MaxwellBoltzmannDistribution(system, temperature_K=300)

calc = MetatomicCalculator("model-llpr.pt")
system.calc = calc

integrator = Bussi(system, timestep=4 * ase.units.fs, temperature_K=300, taut=100 * ase.units.fs)
integrator.run(300)  # equilibrate

nsteps = 1000
energies = []
ensemble_energies = []
integrator.attach(lambda: energies.append(system.get_potential_energy()))
integrator.attach(
    lambda: ensemble_energies.append(calc.run_model(system, outputs={"energy_ensemble": ModelOutput()})["energy_ensemble"].block().values.detach().cpu().numpy())
)

integrator.run(nsteps)

energies = np.array(energies)  # shape (nsteps,)
ensemble_energies = np.concatenate(ensemble_energies)  # shape (nsteps, n_ensemble_members)
thermodynamic_weights = np.exp(-(ensemble_energies - energies[:, None]) / (ase.units.kB * 300))
thermodynamic_weights = thermodynamic_weights / np.sum(thermodynamic_weights, axis=0, keepdims=True)
average_potential_energy_by_ensemble = np.sum(ensemble_energies * thermodynamic_weights, axis=0)  # shape (n_ensemble_members,)

print("Potential energy: ", np.mean(average_potential_energy_by_ensemble))
print("Standard deviation of ensemble average potential energy: ", np.std(average_potential_energy_by_ensemble))

/home/filippo/code/virtualenvs/base/lib/python3.12/site-packages/ase/calculators/calculator.py:538: UserWarning: Some of the atomic energy uncertainties are larger than the threshold of 0.1 eV. The prediction is above the threshold for atoms [ 2 17].
  self.calculate(atoms, [name], system_changes)
/home/filippo/code/virtualenvs/base/lib/python3.12/site-packages/ase/calculators/calculator.py:538: UserWarning: Some of the atomic energy uncertainties are larger than the threshold of 0.1 eV. The prediction is above the threshold for atoms [15 25].
  self.calculate(atoms, [name], system_changes)
/home/filippo/code/virtualenvs/base/lib/python3.12/site-packages/ase/calculators/calculator.py:538: UserWarning: Some of the atomic energy uncertainties are larger than the threshold of 0.1 eV. The prediction is above the threshold for atoms [15 16 25].
  self.calculate(atoms, [name], system_changes)
/home/filippo/code/virtualenvs/base/lib/python3.12/site-packages/ase/calculators/calculator.py:538: 

Potential energy:  1.6013232192993163 1.482823523508977
Standard deviation of ensemble average potential energy:  0.4487590290101564
